In [ ]:
pip install pandas numpy scikit-learn networkx transformers torch nltk

In [12]:
pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.10.0+cpu
Uninstalling torch-2.10.0+cpu:
  Successfully uninstalled torch-2.10.0+cpu
Found existing installation: torchvision 0.25.0+cpu
Uninstalling torchvision-0.25.0+cpu:
  Successfully uninstalled torchvision-0.25.0+cpu
Found existing installation: torchaudio 2.10.0+cpu
Uninstalling torchaudio-2.10.0+cpu:
  Successfully uninstalled torchaudio-2.10.0+cpu
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.10.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.25.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.10.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.10.0%2Bcpu-cp312-cp312-win_amd64.whl (113.7 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.25.0%2Bcpu-cp312-cp312-win_amd64.whl (4.3 MB)
Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.10.0%2Bcpu-cp312-cp312-win_amd64.whl (473 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv("Downloads/archive 2/Drug_related_tweets.csv")
df.head()

,drugName,tweet
0,Mirtazapine,Ive tried a few antidepressants over the years...
1,Mesalamine,My son has Crohns disease and has done very we...
2,Bactrim,Quick reduction of symptoms
3,Contrave,Contrave combines drugs that were used for alc...
4,Cyclafem 1 / 35,I have been on this birth control for one cycl...


In [4]:
df.columns

Index(['drugName', 'tweet'], dtype='object')

In [5]:
df["clean_text"] = df["tweet"].apply(clean_text)

NameError: name 'clean_text' is not defined

In [6]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text

df["clean_text"] = df["tweet"].apply(clean_text)

df.head()

,drugName,tweet,clean_text
0,Mirtazapine,Ive tried a few antidepressants over the years...,ive tried a few antidepressants over the years...
1,Mesalamine,My son has Crohns disease and has done very we...,my son has crohns disease and has done very we...
2,Bactrim,Quick reduction of symptoms,quick reduction of symptoms
3,Contrave,Contrave combines drugs that were used for alc...,contrave combines drugs that were used for alc...
4,Cyclafem 1 / 35,I have been on this birth control for one cycl...,i have been on this birth control for one cycl...


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # limit words for speed
X = vectorizer.fit_transform(df["clean_text"])

In [8]:
df["label"] = df["drugName"].apply(lambda x: 1 if str(x) != "nan" else 0)

y = df["label"]

In [9]:
print("Shape of TF-IDF Matrix:", X.shape)
print("Example feature names:", vectorizer.get_feature_names_out()[:20])

Shape of TF-IDF Matrix: (53766, 5000)
Example feature names: ['aa' 'abdomen' 'abdominal' 'abilify' 'ability' 'ablation' 'able'
 'abnormal' 'abortion' 'about' 'above' 'abreva' 'abruptly' 'abscess'
 'absence' 'absolute' 'absolutely' 'absorb' 'absorbed' 'abuse']


In [10]:
print(df["label"].value_counts())

label
1    53766
Name: count, dtype: int64


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training samples:", X_train.shape)
print("Testing samples:", X_test.shape)

Training samples: (43012, 5000)
Testing samples: (10754, 5000)


In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model training completed")


Model training completed


In [13]:
y_pred = model.predict(X_test)


In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00     10754

    accuracy                           1.00     10754
   macro avg       1.00      1.00      1.00     10754
weighted avg       1.00      1.00      1.00     10754


Confusion Matrix:
 [[10754]]


C:\Users\rahvj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [15]:
df["label"] = df["drugName"].apply(lambda x: 1 if str(x) != "nan" else 0)


In [16]:
# Load normal tweets dataset (non-drug)
normal_df = pd.read_csv("Twitter_Data.csv")

# Keep only the text column
normal_df = normal_df[["clean_text"]]  # if column name different, print(normal_df.columns)

# Assign label 0 = non-drug
normal_df["label"] = 0

# Now drug dataset = label 1
df["label"] = 1

# Combine both datasets
combined_df = pd.concat([df[["clean_text", "label"]], normal_df], ignore_index=True)

combined_df.head()

,clean_text,label
0,ive tried a few antidepressants over the years...,1
1,my son has crohns disease and has done very we...,1
2,quick reduction of symptoms,1
3,contrave combines drugs that were used for alc...,1
4,i have been on this birth control for one cycl...,1


In [17]:
non_drug_samples = [
    "I love watching football on weekends",
    "Weather is so nice today",
    "Studying machine learning is fun",
    "Just had pizza for dinner",
    "Going to the gym now",
    "My laptop battery is draining fast",
    "Traveling to Mumbai tomorrow",
    "Listening to music while coding"
]

non_drug_df = pd.DataFrame({
    "drugName": ["none"] * len(non_drug_samples),
    "tweet": non_drug_samples
})

non_drug_df["clean_text"] = non_drug_df["tweet"].apply(clean_text)
non_drug_df["label"] = 0

In [18]:
drug_df = df.copy()
drug_df["label"] = 1

final_df = pd.concat([drug_df, non_drug_df], ignore_index=True)

In [19]:
X = vectorizer.fit_transform(final_df["clean_text"])
y = final_df["label"]

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [21]:
from sklearn.metrics import classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.999814039981404

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       1.00      1.00      1.00     10753

    accuracy                           1.00     10755
   macro avg       0.50      0.50      0.50     10755
weighted avg       1.00      1.00      1.00     10755


Confusion Matrix:
 [[    0     2]
 [    0 10753]]


C:\Users\rahvj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rahvj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rahvj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
df["label"] = df["clean_text"].apply(
    lambda x: 1 if any(word in x for word in ["buy", "sell", "ship", "cheap", "dm me"]) else 0
)

In [23]:
# Remove rows where clean_text is empty
df = df[df["clean_text"].str.strip() != ""]

# Reset index to keep alignment
df = df.reset_index(drop=True)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["clean_text"])

# Create labels again AFTER cleaning
df["label"] = df["drugName"].apply(lambda x: 1 if str(x) != "nan" else 0)
y = df["label"]

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [27]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     10753

    accuracy                           1.00     10753
   macro avg       1.00      1.00      1.00     10753
weighted avg       1.00      1.00      1.00     10753

[[10753]]


C:\Users\rahvj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [29]:
normal_df = pd.read_csv("tweets.csv")
normal_df["label"] = 0
normal_df.rename(columns={"text": "tweet"}, inplace=True)

drug_df["label"] = 1

df = pd.concat([drug_df, normal_df], ignore_index=True)

In [30]:
print(df['label'].value_counts())

label
1    53766
0    52542
Name: count, dtype: int64


In [31]:
df["clean_text"] = df["tweet"].apply(clean_text)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["clean_text"])
y = df["label"]

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [34]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [35]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10509
           1       1.00      1.00      1.00     10753

    accuracy                           1.00     21262
   macro avg       1.00      1.00      1.00     21262
weighted avg       1.00      1.00      1.00     21262


Confusion Matrix:
 [[10509     0]
 [    0 10753]]


In [36]:
df["label"] = df["drugName"].apply(lambda x: 1 if str(x) != "nan" else 0)

In [37]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["clean_text"])
y = df["label"]

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [39]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10509
           1       1.00      1.00      1.00     10753

    accuracy                           1.00     21262
   macro avg       1.00      1.00      1.00     21262
weighted avg       1.00      1.00      1.00     21262

[[10509     0]
 [    0 10753]]


In [41]:
sample_messages = [
    "Delivery available tonight, contact on private number",
    "Will you come with me for visiting temple",
    "DM for price details, cash only",
    "Enjoying coffee and reading a book"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} ➜ {msg}")


🚨 Suspicious ➜ Delivery available tonight, contact on private number
🚨 Suspicious ➜ Will you come with me for visiting temple
🚨 Suspicious ➜ DM for price details, cash only
🚨 Suspicious ➜ Enjoying coffee and reading a book


In [42]:
normal_samples = [
    "I love watching movies at home",
    "Had dinner with my family",
    "Studying for my exams",
    "Playing cricket with friends",
    "Enjoying coffee in the evening",
    "Listening to music",
    "Went to temple today",
    "Reading a novel",
    "Watching football match",
    "Going to the gym"
] * 50  # repeat to balance

In [43]:
extra_normal_df = pd.DataFrame({
    "tweet": normal_samples,
    "label": 0
})

df = pd.concat([df, extra_normal_df], ignore_index=True)

In [44]:
print(df['label'].value_counts())

label
1    53766
0    53042
Name: count, dtype: int64


In [45]:
normal_samples = [
    "Had a great day at work today!",
    "Let's go to the movies tonight",
    "Happy birthday bro!",
    "Just finished my homework",
    "Family dinner was amazing",
    "Watching cricket match now",
    "Good morning everyone!",
    "Traveling to Chennai tomorrow",
    "Coffee time ☕",
    "Going for gym workout"
]

normal_df = pd.DataFrame({"tweet": normal_samples, "label": 0})
df = pd.concat([drug_df, normal_df], ignore_index=True)

In [46]:
df['clean_text'] = df['clean_text'].fillna('')
df = df[df['clean_text'].str.strip() != '']

In [47]:
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [48]:
sample_messages = [
    "Delivery available tonight, contact on private number",
    "Will you come with me for visiting temple",
    "DM for price details, cash only",
    "Enjoying coffee and reading a book"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

🚨 Suspicious → Delivery available tonight, contact on private number
🚨 Suspicious → Will you come with me for visiting temple
🚨 Suspicious → DM for price details, cash only
🚨 Suspicious → Enjoying coffee and reading a book


In [49]:
import re
import string

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)  # remove links
    text = re.sub(r"@\w+", "", text)     # remove mentions
    text = re.sub(r"#\w+", "", text)     # remove hashtags
    text = re.sub(r"\d+", "", text)      # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [50]:
df["clean_text"] = df["tweet"].apply(clean_text)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),   # captures "private number", "cash only"
    stop_words='english'
)

X = vectorizer.fit_transform(df["clean_text"])
y = df["label"]

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [53]:
sample_messages = [
"Contact me privately, no calls, cash deal only",
"Fast delivery available, message on WhatsApp",
"Package ready, payment in cash only",
"Going to temple with family this evening",
"Watching a movie and eating popcorn",
"Studying for my exams tonight"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

🚨 Suspicious → Contact me privately, no calls, cash deal only
🚨 Suspicious → Fast delivery available, message on WhatsApp
🚨 Suspicious → Package ready, payment in cash only
🚨 Suspicious → Going to temple with family this evening
🚨 Suspicious → Watching a movie and eating popcorn
🚨 Suspicious → Studying for my exams tonight


In [54]:
normal_messages = [
    "Had lunch with my family today",
    "Going to office now",
    "Watching a movie tonight",
    "Studying for my exams",
    "Happy birthday bro",
    "Let’s meet tomorrow at school",
    "Playing cricket with friends",
    "Listening to music and relaxing",
    "Just reached home safely",
    "Going to temple with parents"
]

normal_df = pd.DataFrame({
    "tweet": normal_messages,
    "label": 0
})

In [55]:
drug_df["label"] = 1
df = pd.concat([drug_df[["tweet", "label"]], normal_df], ignore_index=True)

In [56]:
df["clean_text"] = df["tweet"].apply(clean_text)

In [57]:
X = vectorizer.fit_transform(df["clean_text"])
y = df["label"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [58]:
sample_messages = [
    "Contact me privately, no calls, cash deal only",
    "Fast delivery available, message on WhatsApp",
    "Going to temple with family this evening",
    "Watching a movie and eating popcorn",
    "Studying for my exams tonight"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

🚨 Suspicious → Contact me privately, no calls, cash deal only
🚨 Suspicious → Fast delivery available, message on WhatsApp
🚨 Suspicious → Going to temple with family this evening
🚨 Suspicious → Watching a movie and eating popcorn
🚨 Suspicious → Studying for my exams tonight


In [59]:
normal_examples = [
    "Had lunch with my family today",
    "Watching football match tonight",
    "Going to office meeting tomorrow",
    "Listening to my favorite songs",
    "Preparing for exams",
    "Weekend movie night with friends",
    "Going to temple with parents",
    "Attending cousin's wedding",
    "Shopping at the mall",
    "Morning walk was refreshing"
]

normal_df = pd.DataFrame({
    "tweet": normal_examples,
    "label": 0
})

df = pd.concat([df, normal_df], ignore_index=True)


In [60]:
df['clean_text'] = df['tweet'].apply(clean_text)

X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=300)

In [61]:
sample_messages = [
    "Watching a movie and eating popcorn",
    "Going to temple with family this evening",
    "Studying for my exams tonight",
    "Contact me privately, no calls, cash deal only",
    "Fast delivery available, message on WhatsApp"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

✅ Normal → Watching a movie and eating popcorn
✅ Normal → Going to temple with family this evening
✅ Normal → Studying for my exams tonight
🚨 Suspicious → Contact me privately, no calls, cash deal only
🚨 Suspicious → Fast delivery available, message on WhatsApp


In [62]:
sample_messages = [
    "Contact me privately, no calls, cash deal only",
    "Fast delivery available, message on WhatsApp",
    "Going to temple with family this evening",
    "Watching a movie and eating popcorn",
    "Studying for my exams tonight"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

🚨 Suspicious → Contact me privately, no calls, cash deal only
🚨 Suspicious → Fast delivery available, message on WhatsApp
✅ Normal → Going to temple with family this evening
✅ Normal → Watching a movie and eating popcorn
✅ Normal → Studying for my exams tonight


In [63]:
sample_messages = [
    "Delivery available tonight, contact on private number",
    "Will you come with me for visiting temple",
    "DM for price details, cash only",
    "Enjoying coffee and reading a book"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} ➜ {msg}")

🚨 Suspicious ➜ Delivery available tonight, contact on private number
🚨 Suspicious ➜ Will you come with me for visiting temple
🚨 Suspicious ➜ DM for price details, cash only
🚨 Suspicious ➜ Enjoying coffee and reading a book


In [64]:
test_messages = [
    "DM for price and location",
    "Only cash deal, message on WhatsApp",
    "Limited stock available tonight",
    "Contact private number for delivery",
    "Meet near station, don’t bring phone",
    "Special stuff available, hurry up",
    "Fast delivery, cash only",
    "Message me for product details",
]

In [65]:
normal_messages = [
    "Had a great day at the beach",
    "Family dinner was amazing tonight",
    "Studying for exams, feeling tired",
    "Going for a walk in the park",
    "Enjoying coffee and reading a book",
    "Watching a movie with friends",
    "Happy birthday bro, have a great year",
    "Let’s play cricket tomorrow evening",
]

In [66]:
all_test = test_messages + normal_messages

clean_samples = [clean_text(msg) for msg in all_test]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(all_test, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

🚨 Suspicious → DM for price and location
🚨 Suspicious → Only cash deal, message on WhatsApp
🚨 Suspicious → Limited stock available tonight
🚨 Suspicious → Contact private number for delivery
🚨 Suspicious → Meet near station, don’t bring phone
🚨 Suspicious → Special stuff available, hurry up
🚨 Suspicious → Fast delivery, cash only
🚨 Suspicious → Message me for product details
🚨 Suspicious → Had a great day at the beach
🚨 Suspicious → Family dinner was amazing tonight
🚨 Suspicious → Studying for exams, feeling tired
✅ Normal → Going for a walk in the park
🚨 Suspicious → Enjoying coffee and reading a book
✅ Normal → Watching a movie with friends
🚨 Suspicious → Happy birthday bro, have a great year
🚨 Suspicious → Let’s play cricket tomorrow evening


In [67]:
import pandas as pd

behavior_df = pd.read_csv("twitter_Data.csv")
print(behavior_df.columns)

Index(['clean_text', 'category'], dtype='object')


In [68]:
behavior_df['clean_text'] = behavior_df['clean_text'].astype(str).apply(clean_text)

In [69]:
import pandas as pd
import numpy as np

behavior_df = pd.read_csv("twitter_Data.csv")

# Replace missing text with empty string
behavior_df['clean_text'] = behavior_df['clean_text'].fillna("")


In [70]:
# Feature 1: Message length
behavior_df['msg_length'] = behavior_df['clean_text'].apply(len)

# Feature 2: Word count
behavior_df['word_count'] = behavior_df['clean_text'].apply(lambda x: len(x.split()))

# Feature 3: Simulated posting time pattern
behavior_df['hour_of_day'] = np.random.randint(0, 24, size=len(behavior_df))

behavior_df.head()

,clean_text,category,msg_length,word_count,hour_of_day
0,when modi promised “minimum government maximum...,-1.0,210,33,17
1,talk all the nonsense and continue all the dra...,0.0,68,13,8
2,what did just say vote for modi welcome bjp t...,1.0,117,22,14
3,asking his supporters prefix chowkidar their n...,1.0,212,34,14
4,answer who among these the most powerful world...,1.0,81,14,1


In [71]:
behavior_df['category'].isna().sum()

7

In [72]:
behavior_df = behavior_df.dropna(subset=['category'])

In [73]:
behavior_df['category'].isna().sum()

0

In [78]:
sample_messages = [
    "Watching a movie with friends💎",
    "Had lunch with my family today",
    "Enjoying coffee and reading a book",
    "Let’s play cricket tomorrow evening",
    "It's finally snowing! ❄️ Ready to go skiing this weekend? 🎿",
    "The garden is finally looking good! 🌿 I just planted some new shrubs. 🌳 Want to come see?",
    "Yo, you got any 🍃 or 🍀 left? I'm trying to get high tonight. 💨⛽",
    "Party is starting soon, we need more ❄️. Who’s the 🔌 for the 💎 tonight? 🎱",
    "Anyone got 🍫 or 🚌 for tonight? Trying to roll. 🍬⚡❌",
    "Need some 🔮 ASAP. Hit up the 🔌 for the blue 💎. 🧪💙",
    "Family dinner was amazing tonight",
    "The 📦 is landing in 30 ⏰. 💸 sent. 🔌 🪂"
]

clean_samples = [clean_text(msg) for msg in sample_messages]
X_sample = vectorizer.transform(clean_samples)
predictions = model.predict(X_sample)

for msg, pred in zip(sample_messages, predictions):
    label = "🚨 Suspicious" if pred == 1 else "✅ Normal"
    print(f"{label} → {msg}")

✅ Normal → Watching a movie with friends💎
✅ Normal → Had lunch with my family today
🚨 Suspicious → Enjoying coffee and reading a book
🚨 Suspicious → Let’s play cricket tomorrow evening
🚨 Suspicious → It's finally snowing! ❄️ Ready to go skiing this weekend? 🎿
🚨 Suspicious → The garden is finally looking good! 🌿 I just planted some new shrubs. 🌳 Want to come see?
🚨 Suspicious → Yo, you got any 🍃 or 🍀 left? I'm trying to get high tonight. 💨⛽
🚨 Suspicious → Party is starting soon, we need more ❄️. Who’s the 🔌 for the 💎 tonight? 🎱
🚨 Suspicious → Anyone got 🍫 or 🚌 for tonight? Trying to roll. 🍬⚡❌
🚨 Suspicious → Need some 🔮 ASAP. Hit up the 🔌 for the blue 💎. 🧪💙
🚨 Suspicious → Family dinner was amazing tonight
🚨 Suspicious → The 📦 is landing in 30 ⏰. 💸 sent. 🔌 🪂
